In [ ]:
from ultralytics import YOLO
import cv2
from PIL import Image

# Load the YOLOv8 model
model = YOLO("runs/detect/yolov8n_custom/weights/best.pt", task="detection")

In [7]:
# Define the path or URL to the image
image_path = "data/barrel/PXL_20240705_130638462.jpg"

# Run inference on the image
results = model(image_path)

# Get the first result (assuming only one image was passed)
result = results[0]

# Visualize the detections on the image
result_image = result.plot()

# Convert the color space from BGR to RGB
rgb_image = result_image[:, :, ::-1]

# Display the image with detections
Image.fromarray(rgb_image).show()


image 1/1 c:\Users\dk\Desktop\toy-tank-barrel-detection\data\barrel\PXL_20240705_130638462.jpg: 640x480 1 barrel, 71.6ms
Speed: 4.0ms preprocess, 71.6ms inference, 14.0ms postprocess per image at shape (1, 3, 640, 480)


In [8]:
# Get the bounding box coordinates and confidence score of the detected barrel
boxes = result.boxes
conf = boxes.conf
xyxy = boxes.xyxy

# Extract the bounding box coordinates
x1, y1, x2, y2 = xyxy[0]  # Assuming only one barrel is detected

# Calculate the center point (2D position)
center_x = (x1 + x2) / 2
center_y = (y1 + y2) / 2

print(f"Barrel 2D Position: ({center_x:.2f}, {center_y:.2f})")

# Calculate the width and height of the bounding box
width = x2 - x1
height = y2 - y1

# Estimate the orientation based on the aspect ratio
aspect_ratio = width / height
if aspect_ratio > 1:
    orientation = 0  # Horizontal
else:
    orientation = 90  # Vertical

print(f"Barrel Orientation (Aspect Ratio): {orientation}°")

# Refine the orientation estimate using ellipse fitting
# Load the image
image = cv2.imread(image_path)

# Extract the barrel region from the bounding box
barrel_region = image[int(y1):int(y2), int(x1):int(x2)]

# Convert to grayscale and apply thresholding
gray = cv2.cvtColor(barrel_region, cv2.COLOR_BGR2GRAY)
_, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

# Find contours
contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Find the largest contour (assumed to be the barrel)
largest_contour = max(contours, key=cv2.contourArea)

# Fit an ellipse to the largest contour
ellipse = cv2.fitEllipse(largest_contour)
(_, _), (major_axis, _), angle = ellipse

# Adjust the angle to be in the range [0, 180)
refined_orientation = angle if angle < 90 else angle - 180

print(f"Refined Barrel Orientation (Ellipse Fitting): {refined_orientation:.2f}°")

Barrel 2D Position: (740.34, 2213.55)
Barrel Orientation (Aspect Ratio): 0°
Refined Barrel Orientation (Ellipse Fitting): 89.56°
